In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
from   pathlib import Path
import subprocess
import json
import networkx as nx
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

### Function to calculate average number of sequences within 30% identity clusters 

In [3]:
def count_average_number_of_proteins(df, n_select = None):
    used = set()
    Path("./temp/").mkdir(exist_ok=True)
    with open(f"./temp/all_sequences_.fasta",'w') as fo:
        for d_ in df.iloc():
            if d_['pdb_id'] in used:
                continue
            fo.write(f">{d_['pdb_id']}\n{d_['wt_seq']}\n")
            used.add(d_['pdb_id'])
    used = set()
    subprocess.call(f"makeblastdb -in all_sequences_.fasta -dbtype prot", shell=True, cwd="./temp/")
    subprocess.call(f"blastp      -db all_sequences_.fasta -query  all_sequences_.fasta -outfmt 6 -out hits_.tsv -num_threads 4", shell=True, cwd="./temp/")
    hit_data    = pd.read_csv(f"./temp/hits_.tsv", delimiter='\t', header=None)
    hit_data    = hit_data[(hit_data.iloc[:,2]>30.0) & (hit_data.iloc[:,-2]<0.05)]
    G = nx.Graph()
    for h in hit_data.iloc():
        e = [(h.iloc()[0],h.iloc()[1])]
        G.add_edges_from(e)
    subcomponents = list(nx.connected_components(G))
    n_length = []
    output   = []
    for s in subcomponents:
        df_ = df[df["pdb_id"].isin(s)]
        n_length.append(df_.shape[0])
    print(f"Average number of proteins: {np.average(n_length)}")
    
    if n_select is None:
        return np.average(n_length)
    
    sub_df = []
    for s in subcomponents:
        df_ = df[df["pdb_id"].isin(s)]
        df_ = df_.sample(n_select)
        sub_df.append(df_)
    df_concat = pd.concat(sub_df,  ignore_index=True).reset_index(drop=True)
    df_concat.drop(columns=['reverse'], inplace=True)
    return df_concat

### Function to filter out sequences from the training set that share more than 30% identity with the test sequences.


In [4]:
def homology_filter(df_train, 
                                 df_test):
    used = set()
    name = "test_"
    subprocess.call("rm -r temp",shell=True)
    Path("./temp/").mkdir(exist_ok=True)
    with open(f"./temp/all_sequences_{name}.fasta",'w') as fo:
        for d_ in df_train.iloc():
            if d_['pdb_id'] in used:
                continue
            fo.write(f">{d_['pdb_id']}\n{d_['wt_seq']}\n")
            used.add(d_['pdb_id'])
    used = set()
    with open(f"./temp/test_sequences_{name}.fasta",'w') as fo_test:
        for d_ in df_test.iloc():
            if d_['pdb_id'] in used:
                continue
            fo_test.write(f">{d_['pdb_id']}\n{d_['wt_seq']}\n")
            used.add(d_['pdb_id'])
            
    subprocess.call(f"makeblastdb -in all_sequences_{name}.fasta -dbtype prot", shell=True, cwd="./temp/")
    subprocess.call(f"blastp      -db all_sequences_{name}.fasta -query  test_sequences_{name}.fasta -outfmt 6 -out hits_{name}.tsv -num_threads 4", shell=True, cwd="./temp/")
    
    hit_data    = pd.read_csv(f"./temp/hits_{name}.tsv", delimiter='\t', header=None)
    hit_data    = hit_data[(hit_data.iloc[:,2]>30.0) & (hit_data.iloc[:,-2]<0.05)]
    
    G = nx.Graph()
    for h in hit_data.iloc():
        e = [(h.iloc()[0],h.iloc()[1])]
        G.add_edges_from(e)
    subcomponents = list(nx.connected_components(G))    
    all_skip_nodes = set()
    for s in subcomponents:
        all_skip_nodes|=set(s)
    n_before = df_train.shape
    df_train = df_train[~df_train["pdb_id"].isin(all_skip_nodes)]
    n_after  = df_train.shape
    
    print("Size before filtering:",  n_before)
    print("Size after filtering:",   n_after)
    
    return df_train


### Make balanced mix with megadataset

1. Calculate average number of proteins (N) within 30% identity cluster within PROSTATA dataset without megadataset
2. Add proteins N proteins from megadataset for each cluster center to the PROSTATA dataset

In [5]:
df_old       = pd.read_csv("./DATASETS/new_ds_for_s669.csv")
N            = count_average_number_of_proteins(df_old)
s669         = pd.read_csv("./DATASETS/s669.csv")
df_old       = homology_filter(df_old,  s669)
df_mega      = pd.read_csv("./DATA/megadataset/train.csv")
df_mega      = homology_filter(df_mega, s669)
df_mega      = count_average_number_of_proteins(df_mega, n_select = int(N))
mix_dataset  = pd.concat([df_old, df_mega], ignore_index=True).reset_index(drop=True)
datasets     = {}
df_old.to_csv(     "./DATASETS/prostata_ds_no_megadataset_for_s669.csv"    )
mix_dataset.to_csv("./DATASETS/prostata_ds_for_s669.csv"                   )




Building a new DB, current time: 09/18/2023 12:45:59
New DB name:   /Users/ivanisenko/projects/prostata/git_hub_prostata/PROSTATA/temp/all_sequences_.fasta
New DB title:  all_sequences_.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 560 sequences in 0.00458407 seconds.


Average number of proteins: 70.734375


Building a new DB, current time: 09/18/2023 12:46:04
New DB name:   /Users/ivanisenko/projects/prostata/git_hub_prostata/PROSTATA/temp/all_sequences_test_.fasta
New DB title:  all_sequences_test_.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 560 sequences in 0.00410914 seconds.


Size before filtering: (9054, 7)
Size after filtering: (8428, 7)


Building a new DB, current time: 09/18/2023 12:46:21
New DB name:   /Users/ivanisenko/projects/prostata/git_hub_prostata/PROSTATA/temp/all_sequences_test_.fasta
New DB title:  all_sequences_test_.fasta
Sequence ty

### Prepare test set of de novo and natural miniproteins from the megadataset

In [6]:
df_mega = pd.read_csv("DATA/megadataset/train.csv")
df_test = pd.read_csv("DATA/megadataset/test.csv")
df_val  = pd.read_csv("DATA/megadataset/val.csv")
df_all  = pd.concat([df_mega, df_test, df_val])
dfs = []
for d in ["S2648", "S3421", "S3488", "s669", "ssym", "myoglobin", "p53"]:
    dfs.append(pd.read_csv(f"./DATASETS/{d}.csv"))
dfs = pd.concat(dfs)
df_mega_new = homology_filter(df_mega, dfs)
print(df_mega_new.shape)

de_novo_pdb_list = ['HEEH_KT_rd6_0200.pdb', 'EEHEE_rd2_0770.pdb', 'HEEH_rd4_0097.pdb', 'EEHEE_rd3_0144.pdb', 'EEHEE_rd4_0418.pdb', 'HEEH_rd4_0094.pdb', 'EHEE_rd2_0372.pdb', 'EHEE_rd4_0044.pdb', 'EEHEE_rd3_0657.pdb', 'EHEE_rd2_0191.pdb', 'EHEE_rd2_0751.pdb', 'EEHEE_rd3_1587.pdb', 'HEEH_KT_rd6_0793.pdb', 'HEEH_KT_rd6_0790.pdb', 'EHEE_rd4_0172.pdb', 'EEHEE_rd3_1810.pdb', 'EHEE_rd4_0463.pdb', 'EEHEE_rd3_1702.pdb', 'EEHEE_rd3_1817.pdb', 'EEHEE_rd3_1498.pdb', 'EHEE_rd2_0152.pdb', 'EHEE_rd3_0067.pdb', 'HEEH_KT_rd6_3632.pdb', 'HEEH_rd3_0223.pdb', 'EEHEE_rd3_0146.pdb', 'EHEE_rd4_0086.pdb', 'EHEE_rd3_0035.pdb', 'EHEE_rd4_0394.pdb', 'EHEE_rd4_0098.pdb', 'EHEE_rd4_0325.pdb', 'EHEE_rd3_0053.pdb', 'EHEE_rd2_0487.pdb', 'EHEE_rd1_0101.pdb', 'EHEE_rd4_0300.pdb', 'EEHEE_rd3_1558.pdb', 'EEHEE_rd4_0647.pdb', 'EEHEE_rd3_0094.pdb', 'EEHEE_rd4_0003.pdb', 'EHEE_rd2_0196.pdb', 'EEHEE_rd4_0424.pdb', 'HEEH_rd4_0349.pdb', 'HEEH_KT_rd6_1415.pdb', 'HEEH_KT_rd6_0746.pdb', 'EEHEE_rd3_0602.pdb', 'EHEE_rd4_0195.pdb', 'EEHEE_rd4_0481.pdb', 'EHEE_rd2_0601.pdb', 'EHEE_rd4_0864.pdb', 'EEHEE_rd3_1627.pdb', 'EHEE_rd4_0726.pdb', 'EHEE_rd2_1257.pdb', 'EEHEE_rd4_0256.pdb', 'EHEE_rd4_0840.pdb', 'EHEE_rd2_0628.pdb', 'EEHEE_rd3_1367.pdb', 'EEHEE_rd4_0363.pdb', 'EHEE_rd1_0407.pdb', 'EEHEE_rd3_0019.pdb', 'EEHEE_rd4_0469.pdb', 'HEEH_KT_rd6_0007.pdb', 'HEEH_KT_rd6_0280.pdb', 'EEHEE_rd4_0470.pdb', 'EHEE_rd4_0340.pdb', 'EEHEE_rd4_0794.pdb', 'EEHEE_rd3_1058.pdb']
natural_pdb_list = ['6YSE.pdb', '5KPH.pdb', '1F0M.pdb', '2LGW.pdb', '2MA4.pdb', '6M3N.pdb', '6EWS.pdb', '7BPM.pdb', '2L9R.pdb', '3L1X.pdb', '2JT1.pdb', '1AOY.pdb', '2KFV.pdb', '2JWT.pdb', '5JRT.pdb', '2L7F.pdb', '2LC2.pdb', '5UP5.pdb', '2LQK.pdb', '2M8U.pdb', '2N88.pdb', '2RRU.pdb', '2K5P.pdb', '2MI6.pdb', '2JZ2.pdb', '2GP8.pdb', '2KVT.pdb', '2L7M.pdb', '2L6Q.pdb', '3DKM.pdb', '2MC5.pdb', '2WNM.pdb', '1K1V.pdb', '1R69.pdb', '2KWH.pdb', '2M2L.pdb', '2K1B.pdb', '2B88.pdb', '2LHR.pdb', '5LXJ.pdb', '6ACV.pdb', '2LO1.pdb', '2JVD.pdb', '5Z2S.pdb', '2JY8.pdb', '2N7Y.pdb', '1MHN.pdb', '2L09.pdb', '1GJS.pdb', '2OCH.pdb', '1QP2.pdb', '1V1C.pdb', '6EWT.pdb', '2D1U.pdb', '1YG0.pdb', '2L8D.pdb', '2RU9.pdb', '2MYX.pdb', '1WCL.pdb', '2M2J.pdb', '3ZGK.pdb', '1QKH.pdb', '2MCK.pdb', '2M0C.pdb', '2LYR.pdb', '1LP1.pdb', '1PV0.pdb', '6FVC.pdb', '2MPK.pdb', '2LVN.pdb', '1TG0.pdb', '6EWU.pdb', '2LYQ.pdb', '2MXD.pdb', '1IFY.pdb', '2QFF.pdb', '2LT1.pdb', '4G3O.pdb', '2KRU.pdb', '2JRR.pdb', '2MKY.pdb', '2JRO.pdb', '2MWA.pdb', '3V1A.pdb', '2MCH.pdb', '2MRL.pdb', '2M8E.pdb', '2JN4.pdb', '2K28.pdb', '1ENH.pdb']

df_de_novo = df_mega_new[df_mega_new["pdb_id"].isin(de_novo_pdb_list)]
df_natural = df_mega_new[df_mega_new["pdb_id"].isin(natural_pdb_ids)]

df_de_novo[~df_de_novo["reverse"]].to_csv("./DATASETS/miniproteins_denovo_megadataset.csv")
df_natural[~df_natural["reverse"]].to_csv("./DATASETS/miniproteins_natural_megadataset.csv")




Building a new DB, current time: 09/18/2023 12:52:01
New DB name:   /Users/ivanisenko/projects/prostata/git_hub_prostata/PROSTATA/temp/all_sequences_test_.fasta
New DB title:  all_sequences_test_.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 388 sequences in 0.00287604 seconds.


Size before filtering: (735716, 8)
Size after filtering: (493284, 8)
(493284, 8)


NameError: name 'natural_pdb_ids' is not defined

###
### Prepare train and test splits for experiments in the paper




In [8]:
def reverse_dataset(df):
    ### function to add inverse mutations
    df = df.copy()
    w,m = df[ "wt_seq"].copy(),df["mut_seq"].copy()
    df["mut_seq"] =  w
    df["wt_seq"]  =  m
    df["ddg"]     *= -1
    mut_ids = []
    for d_ in df.iloc():
        mut     = d_["mut_info"]
        mut_rev = mut[-1]+mut[1:-1]+mut[0]
        mut_ids.append(mut_rev)
    df["mut_info"]= mut_ids
    return df

In [9]:
datasets                     = {}
datasets["prostata"]         = pd.read_csv("DATASETS/prostata_ds_for_s669.csv")
datasets["hem_test"]         = pd.read_csv("DATASETS/case_study_dimer_test_with_labels.csv"      )
datasets["oligo_test"]       = pd.read_csv("DATASETS/case_study_gem_test_with_labels.csv"        )
datasets["s669_with_dssp"]   = pd.read_csv("DATASETS/s669_with_dssp.csv"                         )
datasets["S2648"]            = pd.read_csv("DATASETS/S2648.csv"         )
datasets["S2648_r"]          = reverse_dataset(pd.read_csv("DATASETS/S2648.csv"                 ))
datasets["Q3421"]            = pd.read_csv("DATASETS/S3421.csv"         )
datasets["Q3488"]            = pd.read_csv("DATASETS/S3488.csv"         )
datasets["Vb"]               = pd.read_csv("DATASETS/ACDC_varibench.csv")
datasets["Vb_r"]             = reverse_dataset(datasets["Vb"]               )
datasets["ssym"]             = pd.read_csv("DATASETS/ssym.csv"          )
datasets["ssym_r"]           = pd.read_csv("DATASETS/ssym_r.csv"        ) 
datasets["s669"]             = pd.read_csv("DATASETS/s669.csv"          )
datasets["s669_r"]           = pd.read_csv("DATASETS/s669_r.csv"        )
datasets["s669_dssp"]        = pd.read_csv("DATASETS/s669_with_dssp.csv"     )
datasets["myoglobin"]        = pd.read_csv("DATASETS/myoglobin.csv"     )
datasets["myoglobin_r"]      = reverse_dataset(datasets["myoglobin"])
datasets["p53"]              = pd.read_csv("DATASETS/p53.csv"           )
datasets["p53_r"]            = reverse_dataset(datasets["p53"])
datasets["mini_nat"]         = pd.read_csv("./DATASETS/miniproteins_natural_megadataset.csv")           
datasets["mini_nat_r"]       = reverse_dataset(datasets["mini_nat"])
datasets["mini_denovo"]     = pd.read_csv("./DATASETS/miniproteins_denovo_megadataset.csv")    
datasets["mini_denovo_r"]   = reverse_dataset(datasets["mini_denovo"])

### Remove sequences from the train set that shares similarity with test set

In [10]:
experiments = [{"train":["S2648","S2648_r","Vb","Vb_r"], "test":["p53","p53_r", 
                                                                 "mini_nat",
                                                                 "mini_denovo"]},
               {"train":["Q3488"],             "test":["ssym",           "ssym_r"]},
               {"train":["Q3488"],             "test":["p53",             "p53_r"]},
               {"train":["Q3488"],             "test":["myoglobin", "myoglobin_r"]},
               {"train":["S2648","S2648_r","Vb","Vb_r"], "test":["myoglobin", "myoglobin_r"]},
               {"train":["Q3421"],             "test":["ssym",           "ssym_r"]},
               {"train":["prostata"],          "test":["s669","s669_r","s669_dssp"]},
               {"train":["prostata"],          "test":["hem_test"]},
               {"train":["prostata"],          "test":["oligo_test"]},
               {"train":["Q3421"],             "test":["ssym", "ssym_r"]}]
    
Path("./PROSTATA_EXPERIMENTS/").mkdir(exist_ok=True)
meta_info = []
for experiment in experiments:
    train_ids, test_ids = experiment["train"], experiment["test"]
    train_set      = pd.concat([datasets[t] for t in train_ids]).reset_index(drop=True)
    train_set_name = ".".join(train_ids)
    test_set_name  = ".".join(test_ids)
    output_path    = "./PROSTATA_EXPERIMENTS/train_"+train_set_name+"_test_"+test_set_name
    Path(output_path).mkdir(exist_ok=True)           
    train_set_filtered = homology_filter(train_set,                                              
                                                datasets[test_ids[0]])
    
    meta_info_ = {"train_set_size_unfiltered":train_set.shape[0],
                  "train_set_size_filtered":  train_set_filtered.shape[0],
                  "test_set_size":            datasets[test_ids[0]].shape[0],
                  "train_set_ids":            train_ids,
                  "test_set_ids" :            test_ids}
    meta_info.append(meta_info_)
    train_set_filtered.to_csv(output_path+"/train.csv")
    for test_id in test_ids:
        datasets[test_id].to_csv(output_path+f"/test_{test_id}.csv")
    



Building a new DB, current time: 09/18/2023 12:53:33
New DB name:   /Users/ivanisenko/projects/prostata/git_hub_prostata/PROSTATA/temp/all_sequences_test_.fasta
New DB title:  all_sequences_test_.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 163 sequences in 0.00183702 seconds.


Size before filtering: (8036, 7)
Size after filtering: (8016, 7)


Building a new DB, current time: 09/18/2023 12:53:35
New DB name:   /Users/ivanisenko/projects/prostata/git_hub_prostata/PROSTATA/temp/all_sequences_test_.fasta
New DB title:  all_sequences_test_.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 127 sequences in 0.00149798 seconds.


Size before filtering: (3488, 7)
Size after filtering: (3458, 7)


Building a new DB, current time: 09/18/2023 12:53:36
New DB name:   /Users/ivanisenko/projects/prostata/git_hub_prostata/PROSTATA/temp/all_sequences_test_.fasta
New DB title:  

In [12]:
for m in meta_info:
    print("/".join(m["train_set_ids"][:2]), "&", m["test_set_ids"][0], "&", m["train_set_size_filtered"],"/",m["train_set_size_unfiltered"], "&", m["test_set_size"])

S2648/S2648_r & p53 & 8016 / 8036 & 42
Q3488 & ssym & 3458 / 3488 & 342
Q3488 & p53 & 3488 / 3488 & 42
Q3488 & myoglobin & 3488 / 3488 & 134
S2648/S2648_r & myoglobin & 7888 / 8036 & 134
Q3421 & ssym & 1812 / 3407 & 342
prostata & s669 & 13678 / 13678 & 669
prostata & hem_test & 13326 / 13678 & 294
prostata & oligo_test & 13320 / 13678 & 382
Q3421 & ssym & 1812 / 3407 & 342
